In [1]:
import entity_tagging
import utils
import argparse
from datetime import datetime
import pandas as pd
import pickle
from sklearn.model_selection import KFold
import numpy as np

In [2]:
def load_corpus_xlsx(filename):
    corpus = pd.read_excel(filename, dtype={'msg': str})
    list_label = corpus['class'].to_list()
    list_content = corpus['msg'].to_list()

    return list_label, list_content

In [3]:
list_label, list_content = load_corpus_xlsx('C:\KhoiNXM\Spam message Vietnamese\Data\message_modified_v1.2.xlsx')

In [4]:
list_content = entity_tagging.entity_tagging(list_content)

In [5]:
classifier_list = ['nb', 'svm', 'dt', 'knn', 'maxent', 'baseline']

# Begin test

In [16]:
test_content = 'Giam gia 30-50% toan bo SP tu 13-16/11 tren toan he thong SR cua FPTshop nhan dip SN 9 nam'.lower()
# test_content_list = ['zxcvbnmasdfghjkl', 'Giam gia 30-50% toan bo SP tu 13-16/11 tren toan he thong SR cua FPTshop nhan dip SN 9 nam. Chi tiet tai www.fptshop.com.vn']

In [17]:
list_label, list_content = load_corpus_xlsx('C:\KhoiNXM\Spam message Vietnamese\Data\message_modified_v1.2.xlsx')

In [9]:
def test(list_content, test_content, list_label):
    content = list_content.copy()
    content.insert(0, test_content)
    content = entity_tagging.entity_tagging(content)
    # list_content_vec, list_len_sms, dictionary = utils.vectorize(content, 'bow')
    list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content[1:], list_label, 'bow')
    clf = utils.build_classifier_svm(list_content_vec, list_label)
    pickle.dump(clf, open(r'C:\KhoiNXM\Spam message Vietnamese\Dev\vietnamese-spam-sms-filtering\models\bow_svm_clf.pkl', 'wb'))
    
    # Reads in saved classification model
    load_clf = pickle.load(open(r'C:\KhoiNXM\Spam message Vietnamese\Dev\vietnamese-spam-sms-filtering\models\bow_svm_clf.pkl', 'rb'))

    prediction = load_clf.predict(list_content_vec[:1])

    return prediction

In [ ]:
for content in test_content_list:
    prediction = test(list_content ,content, list_label)
    print(prediction)

In [18]:
list_content.insert(0, test_content)

In [19]:
list_content = entity_tagging.entity_tagging(list_content)

In [20]:
list_content_vec, list_len_sms, dictionary = utils.vectorize(list_content, 'tfidf')

In [24]:
list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content[1:], list_label, 'bow')

In [21]:
list_content_vec.shape

(6311, 2064)

In [26]:
clf = utils.build_classifier_svm(list_content_vec, list_label)
pickle.dump(clf, open(r'C:\KhoiNXM\Spam message Vietnamese\Dev\vietnamese-spam-sms-filtering\models\bow_svm_clf.pkl', 'wb'))

In [28]:
# Reads in saved classification model
load_clf = pickle.load(open(r'C:\KhoiNXM\Spam message Vietnamese\Dev\vietnamese-spam-sms-filtering\models\tfidf_svm_clf.pkl', 'rb'))

In [29]:
list_content_vec[:1]

array([[ 0.23457868,  0.21122605,  0.23881783, ...,  0.        ,
         0.        , 21.        ]])

In [30]:
load_clf.predict(list_content_vec[:1])

ValueError: X has 2064 features, but LinearSVC is expecting 2063 features as input.

# End test

In [24]:
def train_evaluation(list_label, list_content, vectorize_method, classifier_list):

    if vectorize_method == 'bow':
        list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, 'bow')

    if vectorize_method == 'tfidf':
        list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, 'tfidf')

    for i in classifier_list:
        list_false_positive, list_false_negative, list_true_positive, list_true_negative = \
        utils.kfold_classification(list_content, list_content_vec, list_label, i, 5, vectorize_method)
        # print(list_false_positive)
        # print(list_false_negative)
        # print(list_true_positive)
        # print(list_true_negative)
        print('Evaluation for ' + i.upper())
        utils.evaluation(list_false_positive, list_false_negative, list_true_positive, list_true_negative)
        print('-------------------------------------------------')

In [25]:
def train_entire_dataset(list_content, list_label, vectorize_method):
    list_content_vec, list_label, list_len_sms, dictionary = utils.doc_2_vec(list_content, list_label, vectorize_method)
    print(list_content_vec.shape)
    clf = utils.build_classifier_svm(list_content_vec, list_label)
    pickle.dump(clf, open(r'C:\KhoiNXM\Spam message Vietnamese\Dev\vietnamese-spam-sms-filtering\models\tfidf_svm_clf.pkl', 'wb'))
    print('Traning Done!')

In [26]:
train_entire_dataset(list_content, list_label,'tfidf')

(6311, 2064)


ValueError: Found input variables with inconsistent numbers of samples: [6311, 6310]

In [7]:
train_evaluation(list_label, list_content, 'bow', classifier_list)

Evaluation for NB
False Positive Rate: 1.0097780609258014%
False Negative Rate: 18.796993116465238%
True Positive Rate: 81.20300688353478%
True Negative Rate: 98.99022193907419%
-------------------------------------------------
Evaluation for SVM
False Positive Rate: 1.3351819116360293%
False Negative Rate: 18.89210839845861%
True Positive Rate: 81.10789160154138%
True Negative Rate: 98.66481808836397%
-------------------------------------------------
Evaluation for DT
False Positive Rate: 4.4972973071381%
False Negative Rate: 24.464341943445863%
True Positive Rate: 75.53565805655413%
True Negative Rate: 95.5027026928619%
-------------------------------------------------
Evaluation for KNN
False Positive Rate: 2.611482236328999%
False Negative Rate: 40.57512427389541%
True Positive Rate: 59.42487572610459%
True Negative Rate: 97.388517763671%
-------------------------------------------------
Evaluation for MAXENT
False Positive Rate: 1.239831309480011%
False Negative Rate: 19.175452150

In [8]:
train_evaluation(list_label, list_content, 'tfidf', classifier_list)

Evaluation for NB
False Positive Rate: 0.019230769230769232%
False Negative Rate: 84.59298934396843%
True Positive Rate: 15.40701065603157%
True Negative Rate: 99.98076923076923%
-------------------------------------------------
Evaluation for SVM
False Positive Rate: 0.839462087674848%
False Negative Rate: 28.125645171881505%
True Positive Rate: 71.87435482811848%
True Negative Rate: 99.16053791232515%
-------------------------------------------------
Evaluation for DT
False Positive Rate: 5.010726892903884%
False Negative Rate: 25.6492224273644%
True Positive Rate: 74.3507775726356%
True Negative Rate: 94.98927310709612%
-------------------------------------------------
Evaluation for KNN
False Positive Rate: 2.097489719567876%
False Negative Rate: 50.04580500609026%
True Positive Rate: 49.95419499390973%
True Negative Rate: 97.90251028043213%
-------------------------------------------------
Evaluation for MAXENT
False Positive Rate: 0.7639393369246528%
False Negative Rate: 31.14364